In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
# 1. INSTALL LIBRARIES (Robust Installation)
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.1.0+cpu.html --quiet
!pip install codecarbon --quiet

# 2. IMPORT LIBRARIES AND DEFINE CLASSES
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import copy
import pandas as pd
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool, global_max_pool, global_add_pool
from torch_geometric.nn import BatchNorm, LayerNorm
from codecarbon import EmissionsTracker

def set_seed(seed=42):
    """Sets the seed for reproducibility."""
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GNNSearchSpace:
    """Defines the search space for GNN architectures."""
    def __init__(self):
        self.conv_type = ['GCN', 'GAT', 'SAGE']
        self.pooling_type = ['mean', 'max', 'add']
        self.activation = ['relu', 'elu', 'leaky_relu']
        self.norm_type = ['batch', 'layer', 'none']
        self.hidden_dim = [64, 128]
        self.num_layers = [2, 3]
        self.dropout = [0.2, 0.5]

    def sample_architecture(self):
        return {
            'conv_type': random.choice(self.conv_type), 'pooling_type': random.choice(self.pooling_type),
            'activation': random.choice(self.activation), 'norm_type': random.choice(self.norm_type),
            'hidden_dim': random.choice(self.hidden_dim), 'num_layers': random.choice(self.num_layers),
            'dropout': random.choice(self.dropout)
        }

class GNNModel(nn.Module):
    """A dynamic GNN model built from a configuration dictionary."""
    def __init__(self, config, input_dim, output_dim):
        super(GNNModel, self).__init__()
        self.config = config; self.num_layers = config['num_layers']
        self.convs = nn.ModuleList(); self.norms = nn.ModuleList()
        in_dim, hidden_dim = input_dim, config['hidden_dim']

        for i in range(self.num_layers):
            out_dim = hidden_dim
            if config['conv_type'] == 'GCN': conv = GCNConv(in_dim, out_dim)
            elif config['conv_type'] == 'GAT': conv = GATConv(in_dim, out_dim, heads=1)
            elif config['conv_type'] == 'SAGE': conv = SAGEConv(in_dim, out_dim)
            self.convs.append(conv)
            if i < self.num_layers - 1:
                if config['norm_type'] == 'batch': norm = BatchNorm(out_dim)
                elif config['norm_type'] == 'layer': norm = LayerNorm(out_dim)
                else: norm = nn.Identity()
                self.norms.append(norm)
            in_dim = out_dim

        if config['pooling_type'] == 'mean': self.pool = global_mean_pool
        elif config['pooling_type'] == 'max': self.pool = global_max_pool
        elif config['pooling_type'] == 'add': self.pool = global_add_pool
        self.classifier = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch=None):
        x = x.float()
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            if i < self.num_layers - 1:
                x = self.norms[i](x)
                if self.config['activation'] == 'relu': x = F.relu(x)
                elif self.config['activation'] == 'elu': x = F.elu(x)
                elif self.config['activation'] == 'leaky_relu': x = F.leaky_relu(x)
                x = F.dropout(x, p=self.config['dropout'], training=self.training)
        if batch is not None:
            x = self.pool(x, batch)
        x = self.classifier(x)
        return x

def transfer_weights(parent_model, child_model):
    """Robustly transfers weights between compatible layers."""
    # Transfer convolutional layers
    for i in range(min(len(parent_model.convs), len(child_model.convs))):
        if type(parent_model.convs[i]) == type(child_model.convs[i]) and \
           parent_model.convs[i].in_channels == child_model.convs[i].in_channels and \
           parent_model.convs[i].out_channels == child_model.convs[i].out_channels:
            child_model.convs[i].load_state_dict(parent_model.convs[i].state_dict())

    # --- FIX: Add a shape check before copying normalization layer weights ---
    for i in range(min(len(parent_model.norms), len(child_model.norms))):
        parent_norm = parent_model.norms[i]
        child_norm = child_model.norms[i]
        if type(parent_norm) == type(child_norm) and hasattr(parent_norm, 'weight') and parent_norm.weight.shape == child_norm.weight.shape:
             child_norm.load_state_dict(parent_norm.state_dict())
    return child_model

class CarbonAwareGNNNAS:
    """The main class for the Neural Architecture Search."""
    def __init__(self, dataset_name='MUTAG', population_size=10, generations=5, seed=42):
        set_seed(seed); self.dataset_name = dataset_name
        self.population_size = population_size; self.generations = generations
        self.search_space = GNNSearchSpace(); self.load_dataset()
        self.best_overall_candidate = None

    def load_dataset(self):
        dataset = TUDataset(root=f'data/{self.dataset_name}', name=self.dataset_name)
        self.input_dim = max(dataset.num_node_features, 1)
        self.output_dim = dataset.num_classes
        dataset = dataset.shuffle()
        train_size = int(0.8 * len(dataset))
        self.train_dataset, self.test_dataset = dataset[:train_size], dataset[train_size:]
        self.train_loader = DataLoader(self.train_dataset, batch_size=32, shuffle=True)
        self.test_loader = DataLoader(self.test_dataset, batch_size=32, shuffle=False)
        print(f"Dataset: {self.dataset_name} | Input: {self.input_dim} | Output: {self.output_dim}")

    def calculate_block_reuse(self, arch1, arch2):
        if not arch1 or not arch2: return 0.0
        return sum(1 for k in arch1 if arch1.get(k) == arch2.get(k)) / len(arch1)

    def train_and_evaluate(self, model, parent_model=None, epochs=40):
        if parent_model: model = transfer_weights(parent_model, model)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
        criterion = nn.CrossEntropyLoss()
        tracker = EmissionsTracker(project_name=f"gnn_nas_{self.dataset_name}", log_level='error')
        tracker.start()
        for epoch in range(epochs):
            model.train()
            for batch in self.train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = criterion(out, batch.y); loss.backward(); optimizer.step()
        emissions = tracker.stop()
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.test_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item(); total += batch.y.size(0)
        return {'accuracy': correct / total if total > 0 else 0.0, 'carbon': emissions if emissions else 0.0, 'model': model}

    def mutate_architecture(self, base_config):
        new_config = copy.deepcopy(base_config)
        key_to_mutate = random.choice(list(base_config.keys()))
        options = getattr(self.search_space, key_to_mutate)
        new_config[key_to_mutate] = random.choice(options)
        return new_config

    def tournament_selection(self, population, k=3):
        best = None
        for _ in range(k):
            ind = random.choice(population)
            if best is None or ind['score'] > best['score']: best = ind
        return best

    def run_search(self):
        print("Starting Carbon-Aware GNN NAS...")
        population = []
        
        print("\n--- Training fixed baseline model ---")
        baseline_config = {'conv_type': 'GCN', 'pooling_type': 'mean', 'activation': 'relu', 'norm_type': 'batch', 'hidden_dim': 64, 'num_layers': 2, 'dropout': 0.2}
        baseline_model = GNNModel(baseline_config, self.input_dim, self.output_dim)
        baseline_result = self.train_and_evaluate(baseline_model, epochs=50)
        print(f"  Baseline Results: Acc={baseline_result['accuracy']:.4f}, Carbon={baseline_result['carbon']:.6f} kg")
        
        print("\n--- Generation 0: Initial Population ---")
        for i in range(self.population_size):
            config = self.search_space.sample_architecture()
            model = GNNModel(config, self.input_dim, self.output_dim)
            result = self.train_and_evaluate(model, epochs=40)
            population.append({'config': config, 'parent_config': None, 'score': 0, 'accuracy': result['accuracy'], 'carbon': result['carbon'], 'model': result['model']})
            print(f"  Candidate {i+1}: Acc={result['accuracy']:.4f}, Carbon={result['carbon']:.6f}")
        
        for gen in range(1, self.generations + 1):
            print(f"\n--- Generation {gen}/{self.generations} ---")
            for cand in population:
                reuse_score = self.calculate_block_reuse(cand['config'], cand['parent_config'])
                carbon_score = 1 - min(cand['carbon'] * 1e4, 1.0)
                cand['score'] = 0.6 * cand['accuracy'] + 0.3 * carbon_score + 0.1 * reuse_score
                cand['reuse'] = reuse_score
            population.sort(key=lambda x: x['score'], reverse=True)
            if self.best_overall_candidate is None or population[0]['score'] > self.best_overall_candidate['score']:
                self.best_overall_candidate = population[0]
            print(f"  Best of Gen {gen-1}: Acc={population[0]['accuracy']:.4f}, Carbon={population[0]['carbon']:.6f}, Reuse={population[0]['reuse']:.3f}, Score={population[0]['score']:.4f}")
            next_generation = [population[0]]
            while len(next_generation) < self.population_size:
                parent = self.tournament_selection(population)
                child_config = self.mutate_architecture(parent['config'])
                child_model = GNNModel(child_config, self.input_dim, self.output_dim)
                result = self.train_and_evaluate(child_model, parent_model=parent['model'], epochs=30)
                next_generation.append({'config': child_config, 'parent_config': parent['config'], 'score': 0, 'accuracy': result['accuracy'], 'carbon': result['carbon'], 'model': result['model']})
            population = next_generation

        print("\n--- Search Complete. Final Evaluation ---")
        best_config = self.best_overall_candidate['config']
        print("Retraining best architecture on full data for 100 epochs...")
        final_model = GNNModel(best_config, self.input_dim, self.output_dim)
        final_result = self.train_and_evaluate(final_model, epochs=100)
        
        print("\n--- Final Results ---")
        print(f"Best Architecture: {best_config}")
        print(f"Final Test Accuracy: {final_result['accuracy']:.4f}")
        print(f"Carbon Footprint (Final Train): {final_result['carbon']:.6f} kg CO2")
        print(f"Block Reuse Score (vs. baseline): {self.calculate_block_reuse(best_config, baseline_config):.3f}")

# 3. RUN THE EXPERIMENT
if __name__ == "__main__":
    nas = CarbonAwareGNNNAS(dataset_name='MUTAG', population_size=10, generations=5)
    nas.run_search()

Dataset: MUTAG | Input: 7 | Output: 2
Starting Carbon-Aware GNN NAS...

--- Training fixed baseline model ---
  Baseline Results: Acc=0.6842, Carbon=0.000018 kg

--- Generation 0: Initial Population ---
  Candidate 1: Acc=0.6316, Carbon=0.000012
  Candidate 2: Acc=0.6579, Carbon=0.000016
  Candidate 3: Acc=0.6842, Carbon=0.000014


/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(


  Candidate 4: Acc=0.4474, Carbon=0.000015


/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(


  Candidate 5: Acc=0.6842, Carbon=0.000018
  Candidate 6: Acc=0.6842, Carbon=0.000019
  Candidate 7: Acc=0.7105, Carbon=0.000015
  Candidate 8: Acc=0.6842, Carbon=0.000014
  Candidate 9: Acc=0.6842, Carbon=0.000019


...


--- Generation 4/5 ---
  Best of Gen 3: Acc=0.7368, Carbon=0.000011, Reuse=1.000, Score=0.8089


/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(



--- Generation 5/5 ---
  Best of Gen 4: Acc=0.7368, Carbon=0.000011, Reuse=1.000, Score=0.8089


/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(



--- Search Complete. Final Evaluation ---
Retraining best architecture on full data for 100 epochs...

--- Final Results ---
Best Architecture: {'conv_type': 'SAGE', 'pooling_type': 'add', 'activation': 'elu', 'norm_type': 'batch', 'hidden_dim': 128, 'num_layers': 3, 'dropout': 0.2}
Final Test Accuracy: 0.7105
Carbon Footprint (Final Train): 0.000038 kg CO2
Block Reuse Score (vs. baseline): 0.286
